In [1]:
import torch
from torch.nn.functional import pad

In [2]:
dim = 8
num_heads = 4
# mock embedding matrix 1
seq_len1 = 6
matrix1 = torch.randn(seq_len1, dim)
# mock embedding matrix 2
seq_len2 = 4
matrix2 = torch.randn(seq_len2, dim)
# ensure matrices only contain non-negative values
matrix1 = torch.abs(matrix1)
matrix2 = torch.abs(matrix2)

In [3]:
targets1 = torch.tensor([-1,-1,4,-1,-1,2])
targets2 = torch.tensor([-1,1,-1,8])

In [4]:
targets = [targets1, targets2]

In [5]:
for i, target in enumerate(targets):
    targets[i] = torch.roll(target,-1)
    targets[i][-1] = -1

In [6]:
targets = torch.nested.nested_tensor(targets)

/run/media/john/Secondary/Documents/Cornell/Spring 2024/ORIE 4741 Learning From Data/Project/proj/lib64/python3.11/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  return _nested.nested_tensor(


In [7]:
targets = torch.nested.to_padded_tensor(targets, -1)

In [8]:
targets

tensor([[-1,  4, -1, -1,  2, -1],
        [ 1, -1,  8, -1, -1, -1]])

In [9]:
matrices = [matrix1, matrix2]

In [10]:
longest_seq_len = max([matrix.size(0) for matrix in matrices])

In [11]:
seq_lens = [matrix.size(0) for matrix in matrices]

In [12]:
for i in range(len(matrices)):
    m = matrices[i]
    matrices[i] = pad(m, (0, 0, 0, longest_seq_len - m.size(0)), value=-100000)

In [13]:
matrices[-1][-1] = 1

In [14]:
from torch.nn.functional import scaled_dot_product_attention as sdpa

In [15]:
attn_matrices = []

In [16]:
naive_attn = []

In [17]:
for i in range(len(seq_lens)):
    causal_mask = torch.tril(torch.ones(longest_seq_len, longest_seq_len), diagonal=0)
    causal_mask[:, seq_lens[i]: ] = 0
    attn_matrices.append(causal_mask.bool())

In [18]:
for i in range(len(seq_lens)):
    causal_mask = torch.tril(torch.ones(longest_seq_len, longest_seq_len), diagonal=0)
    naive_attn.append(causal_mask.bool())

In [19]:
attn_matrices = torch.stack(attn_matrices)
naive_attn = torch.stack(naive_attn)

In [20]:
matrices = torch.stack(matrices)

In [21]:
q = torch.rand_like(matrices)
k = torch.rand_like(matrices)

In [22]:
batch_size, _, _ = q.shape

In [23]:
q = q.reshape(batch_size, -1, num_heads, dim // num_heads).transpose(1, 2)
k = k.reshape(batch_size, -1, num_heads, dim // num_heads).transpose(1, 2)
matrices = matrices.reshape(batch_size, -1, num_heads, dim // num_heads).transpose(1, 2)

In [25]:
naive_attn.shape

torch.Size([2, 6, 6])

In [26]:
naive_attn.unsqueeze(1).shape

torch.Size([2, 1, 6, 6])

In [27]:
result = sdpa(q, k, matrices, naive_attn.unsqueeze(1))

In [28]:
result = result.transpose(1, 2).reshape(batch_size, -1, dim)

In [31]:
result

tensor([[[ 6.6287e-01,  1.3786e-01,  2.0032e-01,  2.2771e-01,  1.5119e+00,
           4.4166e-01,  2.5498e-01,  9.5068e-01],
         [ 4.9620e-01,  2.4639e-01,  5.2187e-01,  8.2923e-01,  1.2061e+00,
           7.7659e-01,  5.4411e-01,  8.0955e-01],
         [ 9.7245e-01,  3.5233e-01,  5.0575e-01,  7.1621e-01,  9.9523e-01,
           5.7724e-01,  3.8290e-01,  8.0256e-01],
         [ 9.1522e-01,  3.7262e-01,  5.9163e-01,  7.5641e-01,  8.5354e-01,
           4.9160e-01,  3.0517e-01,  7.6057e-01],
         [ 8.0088e-01,  3.0581e-01,  5.5612e-01,  9.0470e-01,  1.0917e+00,
           5.2244e-01,  4.9384e-01,  7.2574e-01],
         [ 1.0317e+00,  2.8492e-01,  4.6904e-01,  8.9598e-01,  1.0835e+00,
           5.0393e-01,  4.9092e-01,  6.5122e-01]],

        [[ 3.2964e-01,  2.3999e-01,  5.2253e-01,  5.7660e-02,  1.1783e-01,
           1.5077e-01,  2.0024e+00,  2.1966e+00],
         [ 6.5230e-01,  2.0640e-01,  7.9830e-01,  2.8336e-01,  6.3062e-01,
           6.2042e-01,  1.2868e+00,  1.5446e+00]

In [29]:
result.shape

torch.Size([2, 6, 8])

In [ ]:
attn_matrices.shape

torch.Size([2, 6, 6])

In [30]:
result2 = sdpa(q, k, matrices, attn_matrices.unsqueeze(1))

In [32]:
result2 = result2.transpose(1, 2).reshape(batch_size, -1, dim)

In [33]:
result2

tensor([[[0.6629, 0.1379, 0.2003, 0.2277, 1.5119, 0.4417, 0.2550, 0.9507],
         [0.4962, 0.2464, 0.5219, 0.8292, 1.2061, 0.7766, 0.5441, 0.8095],
         [0.9725, 0.3523, 0.5057, 0.7162, 0.9952, 0.5772, 0.3829, 0.8026],
         [0.9152, 0.3726, 0.5916, 0.7564, 0.8535, 0.4916, 0.3052, 0.7606],
         [0.8009, 0.3058, 0.5561, 0.9047, 1.0917, 0.5224, 0.4938, 0.7257],
         [1.0317, 0.2849, 0.4690, 0.8960, 1.0835, 0.5039, 0.4909, 0.6512]],

        [[0.3296, 0.2400, 0.5225, 0.0577, 0.1178, 0.1508, 2.0024, 2.1966],
         [0.6523, 0.2064, 0.7983, 0.2834, 0.6306, 0.6204, 1.2868, 1.5446],
         [0.6084, 0.4371, 1.1620, 0.4250, 0.4935, 0.6620, 1.0227, 1.3305],
         [0.9542, 0.3289, 1.1881, 0.6917, 0.4073, 0.6709, 0.9499, 1.0420],
         [0.9639, 0.2970, 1.1933, 0.7012, 0.3663, 0.6422, 0.9855, 1.0241],
         [0.9745, 0.3334, 1.1888, 0.6978, 0.3546, 0.6417, 0.9073, 0.9793]]])

In [ ]:
before_roll = torch.tensor([0,-1,-1,3,-1,-1,-1,1])

In [ ]:
rolled = torch.roll(before_roll,-1)

In [ ]:
rolled

tensor([-1, -1,  3, -1, -1, -1,  1,  0])

In [ ]:
rolled[-1] = -1

In [ ]:
rolled

tensor([-1, -1,  3, -1, -1, -1,  1, -1])